# Import Libraries

In [1]:
import pandas as pd
import numpy as np

import scipy.stats as scs
import statsmodels.stats.api as sms
from math import ceil

import warnings 
warnings.filterwarnings('ignore')

## Read Dataset

In [2]:
dataset_info = {"filename" : "data/ab.csv",
                "separator" : ","}

In [3]:
df = pd.read_csv(dataset_info['filename'], dataset_info['separator'])

In [4]:
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [6]:
pd.crosstab(df['group'], df['landing_page'])

landing_page,new_page,old_page
group,,
control,1928,145274
treatment,145311,1965


# Data Preparation

## Handling Data Duplication

In [7]:
count_of_sessions = df['user_id'].value_counts(ascending=False)
users_duplicated = count_of_sessions[count_of_sessions > 1].count()

print(f'There are {users_duplicated} observations of user that appear multiple time in the dataset.')

There are 3894 observations of user that appear multiple time in the dataset.


In [8]:
duplicates_to_drop = count_of_sessions[count_of_sessions > 1].index

df = df[~df['user_id'].isin(duplicates_to_drop)]
print(f'After dropping the duplicated data, now weare left with {df.shape[0]} observations.')

After dropping the duplicated data, now weare left with 286690 observations.


## Calculating Conversion Rate by Group

In [9]:
df_group = df.groupby('group').agg({'user_id':'nunique',
                                    'converted':'sum'}).reset_index()
df_group = df_group.rename(columns={'user_id': 'total_users', 'converted': 'conversions'})
df_group['total_users_pct']=(df_group['total_users']/len(df.index))*100
df_group['conversion_pct']=(df_group['conversions']/df['converted'].sum())*100
df_group['conversion_rate']=(df_group['conversions']/df_group['total_users'])*100
df_group

,group,total_users,conversions,total_users_pct,conversion_pct,conversion_rate
0,control,143293,17220,49.981862,50.284713,12.017335
1,treatment,143397,17025,50.018138,49.715287,11.872633


In [10]:
cvr_control = df_group['conversion_rate'][0]
cvr_treatement = df_group['conversion_rate'][1]
cvr_diff = cvr_control-cvr_treatement

In [11]:
print(f'The conversion rate for Control group is {cvr_control:.2f}%, while the treatment is {cvr_treatement:.2f}% with a difference of {cvr_diff:.2f}%.')

The conversion rate for Control group is 12.02%, while the treatment is 11.87% with a difference of 0.14%.


## Calculating Sample

In [12]:
# Calculating effect size based on our expected rates
effect_size = sms.proportion_effectsize(0.13, 0.15)    

# Calculating sample size required based on alpha=0.05 and power=0.8
required_n = sms.NormalIndPower().solve_power(effect_size, 
                                              power=0.8, 
                                              alpha=0.05, 
                                              ratio=1)
# Rounding up to next whole number
required_n = ceil(required_n)                                                 

print(f'The required sample size for the statistical testing is {required_n}.')

The required sample size for the statistical testing is 4720.


## Sampling the Dataset

In [13]:
control_sample = df[df['group'] == 'control'].sample(n=required_n, random_state=22)
treatment_sample = df[df['group'] == 'treatment'].sample(n=required_n, random_state=22)

ab_test_df = pd.concat([control_sample, treatment_sample], axis=0)
ab_test_df.reset_index(drop=True, inplace=True)

In [14]:
ab_test_df

,user_id,timestamp,group,landing_page,converted
0,763854,2017-01-21 03:43:17.188315,control,old_page,0
1,690555,2017-01-18 06:38:13.079449,control,old_page,0
2,861520,2017-01-06 21:13:40.044766,control,old_page,0
3,630778,2017-01-05 16:42:36.995204,control,old_page,0
4,656634,2017-01-04 15:31:21.676130,control,old_page,0
...,...,...,...,...,...
9435,908512,2017-01-14 22:02:29.922674,treatment,new_page,0
9436,873211,2017-01-05 00:57:16.167151,treatment,new_page,0
9437,631276,2017-01-20 18:56:58.167809,treatment,new_page,0
9438,662301,2017-01-03 08:10:57.768806,treatment,new_page,0


In [15]:
ab_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9440 entries, 0 to 9439
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       9440 non-null   int64 
 1   timestamp     9440 non-null   object
 2   group         9440 non-null   object
 3   landing_page  9440 non-null   object
 4   converted     9440 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 368.9+ KB


In [16]:
ab_test_df['group'].value_counts()

control      4720
treatment    4720
Name: group, dtype: int64

In [17]:
df_group_sample = ab_test_df.groupby('group').agg({'user_id':'nunique',
                                                   'converted':'sum'}).reset_index()
df_group_sample = df_group_sample.rename(columns={'user_id': 'total_users', 'converted': 'conversions'})
df_group_sample['total_users_pct']=(df_group_sample['total_users']/len(ab_test_df.index))*100
df_group_sample['conversion_pct']=(df_group_sample['conversions']/ab_test_df['converted'].sum())*100
df_group_sample['conversion_rate']=(df_group_sample['conversions']/df_group['total_users'])*100
df_group_sample

,group,total_users,conversions,total_users_pct,conversion_pct,conversion_rate
0,control,4720,582,50.0,49.531915,0.406161
1,treatment,4720,593,50.0,50.468085,0.413537


## Basic Statistical Check

In [18]:
df_stats = ab_test_df.groupby('group')['converted']

# Std. deviation of the sample
std_p = lambda x: np.std(x, ddof=0)  
# Std. error of the proportion (std / sqrt(n))
se_p = lambda x: scs.sem(x, ddof=0)

df_stats = df_stats.agg([np.mean, std_p, se_p])
df_stats.columns = ['conversion_rate', 'std_deviation', 'std_error']

df_stats

,conversion_rate,std_deviation,std_error
group,,,
control,0.123305,0.328787,0.004786
treatment,0.125636,0.331438,0.004824


In [19]:
cvr_control = (df_stats['conversion_rate'][0])*100
cvr_treatement = (df_stats['conversion_rate'][1])*100
cvr_diff = cvr_control-cvr_treatement

In [20]:
print(f'The conversion rate for Control group is {cvr_control:.2f}%, while the treatment is {cvr_treatement:.2f}% with a difference of {cvr_diff:.2f}%.')

The conversion rate for Control group is 12.33%, while the treatment is 12.56% with a difference of -0.23%.


## Hypothesis Testing

Now, we move on to see if the difference in conversion rate is statsitically significant by performing hypothesis testing.

In [21]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

control_results = ab_test_df[ab_test_df['group'] == 'control']['converted']
treatment_results = ab_test_df[ab_test_df['group'] == 'treatment']['converted']
n_con = control_results.count()
n_treat = treatment_results.count()
successes = [control_results.sum(), treatment_results.sum()]
nobs = [n_con, n_treat]

z_stat, pval = proportions_ztest(successes, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'confidence interval 95% for control group: [{lower_con*100:.2f}%, {upper_con*100:.2f}%]')
print(f'confidence interval 95% for treatment group: [{lower_treat*100:.2f}%, {upper_treat*100:.2f}%]')

z statistic: -0.34
p-value: 0.732
confidence interval 95% for control group: [11.39%, 13.27%]
confidence interval 95% for treatment group: [11.62%, 13.51%]


## Conclusion

In [22]:
def print_conclusion(pval, alpha=0.05):
    if pval>alpha:
        print(f'Since our p-value={pval:.3f} is bigger than alpha={alpha:.3f}, then we must accept the null hypothesis.')
    else:
        print(f'Since our p-value={pval:.3f} is smaller than alpha={alpha:.3f}, then we can reject the null hypothesis.')

In [23]:
print_conclusion(pval, alpha=0.05)

Since our p-value=0.732 is bigger than alpha=0.050, then we must accept the null hypothesis.


In [24]:
pct_diff = ((cvr_treatement-cvr_control)/cvr_control)*100

In [25]:
print(f'The Treatment group has conversion rate of {pct_diff:.2f}% higher than the Control group.')

The Treatment group has conversion rate of 1.89% higher than the Control group.


#### Get Statistics

In [26]:
def get_stats(visitors_A, visitors_B, conversions_A, conversions_B):
    control_cr = conversions_A / visitors_A
    variant_cr = conversions_B / visitors_B
    relative_difference = variant_cr / control_cr - 1
    control_se = (control_cr * (1 - control_cr) / visitors_A) ** 0.5
    variant_se = (variant_cr * (1 - variant_cr) / visitors_B) ** 0.5
    se_difference = (control_se ** 2 + variant_se ** 2) ** 0.5
    return control_cr, variant_cr, relative_difference, control_se, variant_se, se_difference
    

#### Get Power

In [27]:
def get_power(visitors_A, visitors_B, conversions_A, conversions_B, alpha=0.05, two_tails=True):
    control_cr, variant_cr, relative_difference, control_se, variant_se, se_difference = get_stats(visitors_A, visitors_B, conversions_A, conversions_B)
    
    n = visitors_A + visitors_B
    
    if two_tails:
        qu = scs.norm.ppf(1 - alpha / 2)
    else:
        qu = scs.norm.ppf(1 - alpha)

    diff = abs(variant_cr - control_cr)
    avg_cr = (control_cr + variant_cr) / 2

    control_var = control_cr * (1 - control_cr)
    variant_var = variant_cr * (1 - variant_cr)
    avg_var = avg_cr * (1 - avg_cr)

    power_lower = scs.norm.cdf(
        (n ** 0.5 * diff - qu * (2 * avg_var) ** 0.5)
        / (control_var + variant_var) ** 0.5
        )
    power_upper = 1 - scs.norm.cdf(
        (n ** 0.5 * diff + qu * (2 * avg_var) ** 0.5)
        / (control_var + variant_var) ** 0.5
        )

    power = power_lower + power_upper

    return power

#### Get Z Value

In [28]:
def get_z_value(alpha=0.05, two_tails=True):
    z_dist = scs.norm()
    if two_tails:
        alpha = alpha / 2
        area = 1 - alpha
    else:
        area = 1 - alpha

    z = z_dist.ppf(area)
    return z

#### Get Tail Direction

In [29]:
def get_tail_directions(relative_difference, two_tails=True):
    if two_tails is False:
            if relative_difference < 0:
                tail_direction = "right"
            else:
                tail_direction = "left"
    else:
        tail_direction = "two"
    return tail_direction

#### Z_Test

In [30]:
def z_test(visitors_A,visitors_B,conversions_A,conversions_B,alpha,two_tails=True):
        control_cr, variant_cr, relative_difference, control_se, variant_se, se_difference = get_stats(visitors_A, visitors_B, conversions_A, conversions_B)
        tail_direction = get_tail_directions(relative_difference, two_tails)
        
        combined_cr = (conversions_A + conversions_B) / (
            visitors_A + visitors_B
        )
        combined_se = (
            combined_cr
            * (1 - combined_cr)
            * (1 / visitors_A + 1 / visitors_B)
        ) ** 0.5

        # z-score
        z_score = (variant_cr - control_cr) / combined_se

        # Calculate the p-value dependent on one or two tails
        if tail_direction == "left":
            p_value = scs.norm.cdf(-z_score)
        elif tail_direction == "right":
            p_value = scs.norm.cdf(z_score)
        else:
            p_value = 2 * scs.norm.cdf(-abs(z_score))

        return z_score, p_value

#### Get Table Data

In [31]:
def get_table_data(visitors_A,visitors_B,conversions_A,conversions_B,alpha,two_tails=True):

    control_cr, variant_cr, relative_difference, control_se, variant_se, se_difference = get_stats(visitors_A, visitors_B, conversions_A, conversions_B)
    power = get_power(visitors_A, visitors_B, conversions_A, conversions_B)
    z_score, p_value = z_test(visitors_A,visitors_B,conversions_A,conversions_B,alpha,two_tails)
    
    data = {"<b>Variant</b>": ["A", "B"],
            "<b>Visitors</b>": [f"{visitors_A:,}", f"{visitors_B:,}"],
            "<b>Conversions</b>": [conversions_A, conversions_B],
            "<b>Conversion rate</b>": [f"{control_cr:.2%}", f"{variant_cr:.2%}"],
            "<b>Uplift</b>": ["", f"{relative_difference:.2%}"],
            "<b>Power</b>": ["", f"{power:.4f}"],
            "<b>Z-score</b>": ["", f"{z_score:.4f}"],
            "<b>P-value</b>": ["", f"{p_value:.4f}"],
           }
    return data

#### Get Table for Dashboard

In [32]:
def get_table_dash(visitors_A,visitors_B,conversions_A,conversions_B,alpha,two_tails=True):
        
    # get stats, z, power, tail direction
    control_cr, variant_cr, relative_difference, control_se, variant_se, se_difference = get_stats(visitors_A, visitors_B, conversions_A, conversions_B)
    power = get_power(visitors_A, visitors_B, conversions_A, conversions_B)
    z_score, p_value = z_test(visitors_A,visitors_B,conversions_A,conversions_B,alpha,two_tails)
    
    result_dict = {'Control':{'Visitors':visitors_A,
                          'Conversions':conversions_A,
                          'Conversion Rate':control_cr,
                          'Uplift':'',
                          'Power':'',
                          'Z-score':'',
                          'P-value':''},
                   'Treatment':{'Visitors':visitors_B,
                                'Conversions':conversions_B,
                                'Conversion Rate':variant_cr,
                                'Uplift':relative_difference,
                                'Power':power,
                                'Z-score':z_score,
                                'P-value':p_value}}
    return result_dict

## Visualizations within the Notebook

In [33]:
visitors_A = control_sample['user_id'].nunique()
visitors_B = treatment_sample['user_id'].nunique()
conversions_A = control_sample['converted'].sum()
conversions_B = treatment_sample['converted'].sum()

In [34]:
alpha = 0.05
two_tails = True

## Data for Dashboard

## Result Table

In [35]:
result_dict = get_table_dash(visitors_A,visitors_B,conversions_A,conversions_B,alpha,two_tails)
result_df = pd.DataFrame.from_dict(result_dict, orient='index')

In [36]:
result_df

,Visitors,Conversions,Conversion Rate,Uplift,Power,Z-score,P-value
Control,4720,582,0.123305,,,,
Treatment,4720,593,0.125636,0.0189,0.077354,0.342956,0.731632


### Download Results

In [37]:
result_df.to_csv('data/result/ab_result_table.csv')